In [1]:
!pip install openai colorama

In [2]:
!pip install langchain==0.3.23
!pip install langchain-openai==0.3.14
!pip install langchain-community==0.3.21
!pip install langgraph==0.3.31
!pip install langgraph-checkpoint-sqlite==2.0.6
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from openai import OpenAI
from google.colab import userdata
from textwrap import dedent
from getpass import getpass


TAVILY_API_KEY = getpass('Enter Tavily API Key: ')
OPENAI_API_KEY = getpass('Enter OPENAI API Key: ')

import os


os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


tavily_search = TavilySearchAPIWrapper()
!pip install tavily-python

from tavily import TavilyClient
from typing import List, Tuple

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

class ResearchAgent:
    def __init__(self, topic: str, llm_model: str = "gpt-4-turbo"):
        self.topic = topic
        self.model = llm_model
        self.questions = []
        self.qa_pairs = []

    # Step 1: Use LLM to Generate Research Questions
    def generate_questions(self) -> List[str]:
        prompt = f"""
Generate the top 5 searches "{self.topic}".
Ensure they cover rate, state, eligibility crietria, APY.
"""
         # Create an OpenAI client instance
        client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        # Generate response using chat.completions.create
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )

        content = response.choices[0].message.content
        self.questions = [q.strip("0123456789. ") for q in content.split('\n') if q.strip()]
        return self.questions

    # Step 2: Tavily Web Search for Each Question
    def search_answers(self) -> List[Tuple[str, str]]:
        for question in self.questions:
            print(f"🔍 Searching: {question}")
            results = tavily_client.search(query=question, search_depth="advanced", max_results=3)
            content_parts = [
                f"**{res['title']}**\n{res['content']}" for res in results.get("results", [])
            ]
            answer = "\n\n".join(content_parts) if content_parts else "No data found."
            self.qa_pairs.append((question, answer))
        return self.qa_pairs

    # Step 3: Compile Final Report
    def compile_report(self) -> str:
        intro = f"This report explores key questions related to **{self.topic}**, using current online information.\n\n"
        sections = ""
        for idx, (q, a) in enumerate(self.qa_pairs):
            sections += f"### {idx+1}. {q}\n{a}\n\n"
        conclusion = "This concludes the research report. The information above reflects current findings from web sources."
        report = f"# Research Report: {self.topic.title()}\n\n{intro}{sections}---\n\n**Conclusion**\n\n{conclusion}"
        return report

    # Orchestrator
    def run(self):
        self.generate_questions()
        self.search_answers()
        #return self.compile_report()


# 🔧 Example Run
if __name__ == "__main__":
    topic = "The top 5 highest CD rates currently offered by U.S. credit unions in year 2025"
    agent = ResearchAgent(topic=topic, llm_model="gpt-4o")
    report = agent.run()
    print("\n📄 Final Report:\n")
    print(report)


Enter Tavily API Key: ··········
Enter OPENAI API Key: ··········
🔍 Searching: As of 2025, finding the highest CD (Certificate of Deposit) rates offered by U.S. credit unions involves considering the rates, states, eligibility criteria, and annual percentage yields (APY). While I can’t provide real-time data, I can offer a hypothetical list based on trends and typical offerings to aid in your search:
🔍 Searching: **ABC Credit Union**
🔍 Searching: - **APY:** 5.50%
🔍 Searching: - **State:** California
🔍 Searching: - **Eligibility Criteria:** Open to residents of California or employees of select partner companies
🔍 Searching: - **Term:** 3-year CD
🔍 Searching: **First Community Credit Union**
🔍 Searching: - **APY:** 5.45%
🔍 Searching: - **State:** Texas
🔍 Searching: - **Eligibility Criteria:** Must live, work, or worship in Texas or be affiliated with specific Texas organizations
🔍 Searching: - **Term:** 5-year CD
🔍 Searching: **Great Lakes Federal Credit Union**
🔍 Searching: - **APY:** 